In [ ]:
!pip install transformers datasets transformers[torch]

In [ ]:
!pip install -U accelerate

In [ ]:
!pip install rouge.score nltk py7zr

In [ ]:
import transformers
from datasets import load_dataset, load_metric
import numpy as np
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
data = load_dataset("cnn_dailymail",'3.0.0')
metric = load_metric('rouge')
model_checkpoints = 'facebook/bart-large-xsum'

<ipython-input-2-8d6238c7615d>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('rouge')


In [ ]:
max_input = 512
max_target = 128
tokenizer = transformers.AutoTokenizer.from_pretrained(model_checkpoints)

In [ ]:
data['train'] = data['train'].shuffle(seed=123).select(range(4000))
data['validation'] = data['validation'].shuffle(seed=123).select(range(2000))
data['test'] = data['test'].shuffle(seed=123).select(range(800))

In [ ]:
def preprocess_data(data_to_process):
  #get the article text
  inputs = [article for article in data_to_process['article']]
  #tokenize text
  model_inputs = tokenizer(inputs,  max_length=max_input, padding='max_length', truncation=True)

  #tokenize labels
  with tokenizer.as_target_tokenizer():
    targets = tokenizer(data_to_process['highlights'], max_length=max_target, padding='max_length', truncation=True)

  model_inputs['labels'] = targets['input_ids']
  #reuturns input_ids, attention_masks, labels
  return model_inputs

In [ ]:
tokenize_data = data.map(preprocess_data, batched = True, remove_columns=['id', 'article', 'highlights'])

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3660: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

In [ ]:
train_sample = tokenize_data['train']
validation_sample = tokenize_data['validation']
test_sample = tokenize_data['test']

In [ ]:
model = transformers.AutoModelForSeq2SeqLM.from_pretrained(model_checkpoints)

In [ ]:
batch_size = 4

In [ ]:
collator = transformers.DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
def compute_rouge(pred):
  predictions, labels = pred
  #decode the predictions
  decode_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
  #decode labels
  decode_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  #compute results
  res = metric.compute(predictions=decode_predictions, references=decode_labels, use_stemmer=True)
  #get %
  res = {key: value.mid.fmeasure * 100 for key, value in res.items()}

  pred_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
  res['gen_len'] = np.mean(pred_lens)

  return {k: round(v, 4) for k, v in res.items()}

In [ ]:
args = transformers.Seq2SeqTrainingArguments(
    'text-summ',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size= 1,
    gradient_accumulation_steps=2,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=3,
    predict_with_generate=True,
    eval_accumulation_steps=1,
    fp16=True
    )

In [ ]:
trainer = transformers.Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenize_data['train'],
    eval_dataset=tokenize_data['validation'],
    data_collator=collator,
    tokenizer=tokenizer,
    compute_metrics=compute_rouge
)

In [ ]:
trainer.train()

You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,0.967400,1.004524,41.427100,19.186300,28.671300,38.413900,61.815000
